## Gerando box de interesse

In [11]:
import geopandas as gpd
import pandas as pd
from IPython.display import clear_output, display

In [2]:
areas = []
areas.append(gpd.read_file('zip://downloads/fwdenviodosshapefilesestudodecaladasdapompiae.zip!zona od pompeia.shp'))
areas.append(gpd.read_file('zip://downloads/fwdenviodosshapefilesestudodecaladasdapompiae.zip!zona od vila leopoldina.shp'))
gdf_areas = gpd.GeoDataFrame(pd.concat(areas))

In [3]:
print(gdf_areas.unary_union.bounds)

(322278.10529588326, 7396095.722869624, 328267.5011342565, 7397708.321301874)


## Calculando Tiles para um zoom de 14

In [4]:
import mercantile

In [5]:
gdf_areas.to_crs(epsg=4326).unary_union.bounds

(-46.74082388541877,
 -23.536289991520956,
 -46.68216990624233,
 -23.521244967840218)

In [6]:
tiles = [t for t in mercantile.tiles(*gdf_areas.to_crs(epsg=4326).unary_union.bounds, zooms=14)]


In [7]:
gpd.GeoDataFrame.from_features([mercantile.feature(t) for t in tiles])#.to_file('tmp/tiles.gpkg', driver='GPKG')

,geometry,title
0,"POLYGON ((-46.75781 -23.52370, -46.75781 -23.5...","XYZ tile Tile(x=6064, y=9293, z=14)"
1,"POLYGON ((-46.75781 -23.54385, -46.75781 -23.5...","XYZ tile Tile(x=6064, y=9294, z=14)"
2,"POLYGON ((-46.73584 -23.52370, -46.73584 -23.5...","XYZ tile Tile(x=6065, y=9293, z=14)"
3,"POLYGON ((-46.73584 -23.54385, -46.73584 -23.5...","XYZ tile Tile(x=6065, y=9294, z=14)"
4,"POLYGON ((-46.71387 -23.52370, -46.71387 -23.5...","XYZ tile Tile(x=6066, y=9293, z=14)"
5,"POLYGON ((-46.71387 -23.54385, -46.71387 -23.5...","XYZ tile Tile(x=6066, y=9294, z=14)"
6,"POLYGON ((-46.69189 -23.52370, -46.69189 -23.5...","XYZ tile Tile(x=6067, y=9293, z=14)"
7,"POLYGON ((-46.69189 -23.54385, -46.69189 -23.5...","XYZ tile Tile(x=6067, y=9294, z=14)"


## Download dos Tiles de Vetor

In [8]:
import mytokens # Keep the secrets saved in ignored file
import mapbox_vector_tile
from urllib.request import urlopen
from vt2geojson.tools import vt_bytes_to_geojson

In [9]:
# Mapilary Vector Tiles URLs
# https://www.mapillary.com/developer/tiles-documentation/

z, x, y = 0, 0, 0

map_features_points_url =  f'https://tiles.mapillary.com/maps/vtp/mly_map_feature_point/2/' #{z}/{x}/{y}
map_features_traffic_signs_url = f'https://tiles.mapillary.com/maps/vtp/mly_map_feature_traffic_sign/2/' #{z}/{x}/{y}
map_computed_coverage = f'https://tiles.mapillary.com/maps/vtp/mly1_computed_public/2/' #{z}/{x}/{y}

In [12]:
points, signs, computed = [], [], []

for x, y, z in [t for t in tiles]:

    clear_output(wait=True)
    print(f'Processando Tile {x}/{y}/{z}')

    with urlopen(map_features_points_url + f'{z}/{x}/{y}?access_token={mytokens.access_token}') as response:
        pbf = response.read()
        points.append(gpd.GeoDataFrame.from_features(vt_bytes_to_geojson(pbf, x, y, z)))

    with urlopen(map_features_traffic_signs_url + f'{z}/{x}/{y}?access_token={mytokens.access_token}') as response:
        pbf = response.read()
        signs.append(gpd.GeoDataFrame.from_features(vt_bytes_to_geojson(pbf, x, y, z)))    

    with urlopen(map_computed_coverage + f'{z}/{x}/{y}?access_token={mytokens.access_token}') as response:
        pbf = response.read()
        computed.append(gpd.GeoDataFrame.from_features(vt_bytes_to_geojson(pbf, x, y, z)))  
    # break

Processando Tile 6067/9293/14


In [97]:
df_signs = pd.concat(signs)
df_signs['first_seen_at'] = pd.to_datetime(df_signs.first_seen_at, unit='ms')
df_signs['last_seen_at'] = pd.to_datetime(df_signs.last_seen_at, unit='ms')
gdf_signs = gpd.GeoDataFrame(df_signs)

In [99]:
df_points = pd.concat(points)
df_points['first_seen_at'] = pd.to_datetime(df_points.first_seen_at, unit='ms')
df_points['last_seen_at'] = pd.to_datetime(df_points.last_seen_at, unit='ms')
gdf_points = gpd.GeoDataFrame(df_points)

In [ ]:
df_computed = pd.concat(computed)
df_computed['first_seen_at'] = pd.to_datetime(df_computed.first_seen_at, unit='ms')
df_computed['last_seen_at'] = pd.to_datetime(df_computed.last_seen_at, unit='ms')
gdf_computed = gpd.GeoDataFrame(df_computed)

In [102]:
gdf_signs.to_file('resultados/sinalizacao-pompeia-vila-leopoldina.gpkg', driver='GPKG', crs=4326)
gdf_points.to_file('resultados/points-pompeia-vila-leopoldina.gpkg', driver='GPKG', crs=4326)

/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [120]:
gdf_signs.loc[gdf_signs.value.str.contains('no-parking'), 'value'].value_counts()

regulatory--no-parking--g5                   2205
regulatory--no-parking--g9                      6
regulatory--no-parking-or-no-stopping--g3       5
regulatory--no-parking--g2                      3
regulatory--no-parking-or-no-stopping--g4       1
regulatory--end-of-no-parking--g1               1
regulatory--no-parking-or-no-stopping--g1       1
Name: value, dtype: int64